## Porous Medium Equation Solver Result Summary (1D)

**Woojeong Kim** *7/30/2025*

    

### Porous Medium Equation
The **porous medium equation** is also called as **non-linear heat equation** and  has the form on 1-dimensional domain:
$$
\frac{\partial u(x,t)}{\partial t} = \Delta u(x,t)^m , \text{~ where ~} m > 0 \text{~and~} x \in \Omega.
$$
Main driving force of this equation is Energy term - i.e, the output solution variable $u$ behaves mainly by Energy of the form:
$$
E(u) = \int_{\Omega} u^m du
$$
To analyze the Energy, we take derivative as:
$$
\frac{d E(u)}{dx} = m u^{m-1}u_x 
$$
<table>
  <tr>
    <td align="center">
      <img src="../Heat/Energy_adaptive_std.075/exact_solution.png" width="750"/><br/>
      <small>Figure 1.2: Energy adaptive</small>
    </td>
  </tr>
</table>

### Barenblatt-Kompaneets-Zeldovich similarity solution
The exact solution named as `Barenblatt-Kompaneets-Zeldovich solution` has the form:
$$
u_{\text{exact}}(x, t) = \frac{1}{t^\alpha} \left( b - \frac{m-1}{2m} \beta \frac{\| x \|^2}{t^{2\beta}} \right) _{+}^{\frac{1}{m-1}}
$$
where $x \in \mathbb{R}^n$, $\| \cdot \|^2$ is the $l^2$- norm, $(\cdot)_+$ is the positive part, and
$$
\alpha = \frac{n}{n(m-1) +2}, ~~~~\beta = \frac{1}{n(m-1)+2}
$$

### Physics-Informed Neural Network Loss Term
As the preliminary famework for building PINN, we use the following loss terms to optimize total loss by `.backward()` at the end of each training iteration. The 1-dimensional spatial domain $\Omega := [-5, 5]$ and time domain is $[0,T]$ where $T$ is ending time.
- Loss on domain
  $$
  L_{p} = u(x,t)_tt - u(x,t)^m, ~~~~ (x, t) \in [-5, 5] \times [0, T]
  $$
- Loss derived from initial condition
  $$
  L_i = u_{\text{exact}}(x,.01),  ~~~~ x \in [-5, 5]
  $$
- Loss derived from boundary condition - periodic boundary
  $$
  L_{b1} = u(-5,t) - u(5,t)
  $$
  $$
  {L}_{b2} = u_x(-5,t) - u_x(5,t),  ~~~~ t \in [0, T]
  $$
And, the basic total loss is set as follows.
$$
\implies L_t = L_{p} + 1000 L_i + L_{b1} + L_{b2}
$$

### Experiment 1
This is the base line experiment using the basic total loss $L_t$. In this notebook we use 6 hidden layers with neuron size 100 for the `forward` pass of the neural network and `torch.tanh`activation function on each layer. In this experiment, 12000, 150 and 200 collocation points collocation points were used for computing $L_p$, $L_{b1}+{L}_{b2}$ and $L_i$ respectively.


### Experiment 2
This is the classical experiment for observing the efficiency of adaptive resampling. we splited the basic total loss $L_t$ into the basic loss term and resampled loss term according to the upper 10% high-loss points:
$$
\implies L_t = L_{p1} + L_{p2} + 1000 (L_{i1} + L_{i2}) + L_{b1} + L_{b2}
$$
where $L_{p1} = L_{p}$, $L_{i1} = L_{i}$. And, $L_{p2}$, $L_{i2}$ are pde loss term and initial loss term from the high-loss resampling for the upper 10% from each term. The collocation points numbers are 8000 for $L_{p1}$, 4000 for $L_{p2}$, 150 for $L_{i1}$, 50 for $L_{i2}$ and 150 for $L_{b1} + L_{b2}$.

### Experiment 3
This is the new resampling method in MCMC process, **likelihood-free MCMC**, which is inspired by Metropolis Hasting and probability density approximation to the gradient of eneery $\frac{\partial E}{\partial x}$. By employing this method, we give more training collocation points on the rapidley changing domain area powered by fast change of energy value. Similar to the Experiment2, to compare this with the previous experiment, we splited the basic total loss $L_t$ in experiment 1 into the basic loss term and loss term resampled from this likelihood-free MCMC:
$$
\implies L_t = L_{p1} + 100L_{p3} + 1000 (L_{i1} + L_{i3}) + L_{b1} + L_{b2}
$$
where $L_{p1} = L_{p}$, $L_{i1} = L_{i}$. And, $L_{p3}$, $L_{i3}$ are pde loss term and initial loss term from the likelihood-free MCMC. The collocation points numbers are 8000 for $L_{p1}$, 4000 for $L_{p3}$, 150 for $L_{i1}$, 50 for $L_{i3}$ and 150 for $L_{b1} + L_{b2}$.

### Algorithm: Likelihodd-Free MCMC(LF Metropolis-Hastings) sampling

We are given data $\theta = {\theta_1, \theta_2, ..., \theta_n}$ considered as likelihood of $x = {x_1, x_2, ..., x_n}$. We assume that the likelihood $\theta$ is generalized as $x$ by using KDE likelihood on $\theta$ and neighborhood points for each of $\theta$. With this condition, we will accept or reject proposal $(\theta', x')$ by using acceptance ratio $\alpha$ where $\theta' = {\theta_1', \theta_2', ..., \theta_n'}$. Since $\theta :=  From the current algorithm sate $(\theta, x)$, a new parameter vetor $\theta '$ is drawn from a proposal distribution $q(\theta ' | \theta)$.

**Input**:  
- Target distribution $\pi(x, t) := |u(x, t) u_x(x, t)|$ (Normalized $\frac{d E(u)}{dx} = m u^{m-1}u_x$)  
- Proposal distribution $q(x' | x) ~ N(\pi(x,t), 0.075)$  
- Initial value $x_0 := \pi (x,0)$  
- Total number of approximating sampling iterations $N := 75$

**Output**:  
- Samples $\hat{\theta} = {\hat{\theta}_1, \hat{\theta}_2, ..., \hat{\theta}_n}$ approximating $\pi(x, t)$

**Steps**:

1. Initialize k = 1 and observed data y  ← $\pi(\theta)$ from given $\theta$  
2. For step $k$ with given $(\theta_{k},x_{k}) = (\theta,x)$:
    - (1) Extract generalized simulation data $x_i$ ←  $\pi( B )$ for i = 1,2,...,n where B := {100 points in the closed disk centered $\theta_i$ with radius 0.25}.  
    - (2) Generate $\theta'$ ∼ $q( \theta' | \theta)$
    - (3) Similar to (1), generate generalized simulation data $x_i'$ ← $\pi( B' )$ for i = 1,2,...,n where B' := {100 points in the closed disk centered $\theta_i'$ with radius 0.25}.
    - (4) Compute acceptance ratio:  
      $$ \alpha = \min\left(1, \frac{\pi_{\epsilon}(y|x', \theta') \pi(\theta')q(\theta | \theta')}{\pi_{\epsilon}(y|x, \theta) \pi(\theta)q(\theta' | \theta)}\right) $$
      where $\pi_{\epsilon}(y|x, \theta) = 1$ where $|\text{KDE}(y, x)| < \epsilon$, 0 otherwise for KDE() function computing KDE likelihood for a query y against group x of 100 neighborhood points of the query $\theta$.
    - (5) With probability $\alpha$, accept $(\theta_{k+1},x_{k+1}) = (\theta', x')$ ; else, keep $(\theta,x)$.
3. Increment $k = k+1$ and go to 2. until $k = 75$.


### Resampling Figures for each end time in time slice
<table>
  <tr>
    <td align="center">
      <img src="Loss_adaptive/2by10_adap.png" width="750"/><br/>
      <small>Figure 2: Loss adaptive</small>
    </td>
    <td align="center">
      <img src="Energy_adaptive_std.075/Energy_adaptive_MCMC_resampling_each_time.png" width="750"/><br/>
      <small>Figure 3: Energy adaptive</small>
    </td>
  </tr>
</table>

### Plot Results
<table>
  <tr>
    <td align="center">
      <img src="Loss_adaptive/3D_7_25_18h_end1.2.png" width="750"/><br/>
      <small>Figure 1.1: Loss adaptive</small>
    </td>
    <td align="center">
      <img src="Energy_adaptive_std.075/3D_7_28_7h_end1.2.png" width="750"/><br/>
      <small>Figure 1.2: Energy adaptive</small>
    </td>
  </tr>
</table>

<table>
  <tr>
    <td align="center">
      <img src="Loss_adaptive/2D_7_25_18h_end1.2_2d.png" width="850"/><br/>
      <small>Figure 2.1: Loss adaptive</small>
    </td>
  </tr>
  <tr>
    <td align="center">
      <img src="Energy_adaptive_std.075/2D_7_28_7h_end1.2_2d.png" width="850"/><br/>
      <small>Figure 2.2: Energy adaptive</small>
    </td>
  </tr>
</table>


### Error Value Results

| Error Type        | Experiment 1 : Baseline | Experiment 2 : Loss Adpative | Experiment 3 : Energy Adpative |
|-------------------|--------------------|--------------------|--------------------|
| L∞ L∞ Error (u)      | value₁₁           | 0.46           | 0.44          |
| L∞ Error (u)     | value₂₁           | 0.002141  (relative 0.020912)      | 0.001767  (relative 0.017264) |
| L2 Error (u)      | value₃₁           |  0.002421 (relative 0.016163) | 0.002047 (relative 0.013662) |





### Reference

[1] https://en.wikipedia.org/wiki/Porous_medium_equation  
[2] Steve Brooks, Andrew Gelman, Galin Jones, and Xiao-Li Meng (eds.). *Handbook of Markov Chain Monte Carlo*. Chapman & Hall/CRC, 2011.


